In [51]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Import data
ffdaily = pd.read_csv('ff6_daily_factors.csv')
ffmonthly = pd.read_csv('ff6_monthly_factors.csv')
factorlist = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM']

### Q1 (a)-(f) for all six factors.

In [52]:
factordaily_res = pd.read_csv('ff6_daily_factors.csv', usecols = [0])

for factor in factorlist:
    factordaily = ffdaily[['date', factor]].copy()
    # Calculate rolling variance
    factordaily['rolling_var_'+factor] = factordaily[factor].rolling(window=22, min_periods=22).var()
    # Calculate factor weight
    targetvol = factordaily['rolling_var_'+factor].mean()
    factordaily['weight_'+factor] = targetvol/factordaily['rolling_var_'+factor]
    # Collapse to Monthly
    factordaily['mdate'] = (factordaily['date']/100).astype(int)
    factordaily = factordaily.drop_duplicates('mdate', keep='last')
    
    factordaily = factordaily[['mdate', 'weight_'+factor, 'rolling_var_'+factor]].copy()
    factordaily = factordaily.rename(columns={'mdate':'date'})
    factordaily_res = pd.merge(factordaily_res, factordaily, on='date', how='right')

# Merge weights
factormonthly = pd.merge(ffmonthly, factordaily_res, on='date', how='left')


for factor in factorlist:
    
    # Calculate factor return
    factormonthly['Ret_'+factor] = factormonthly['weight_'+factor].shift(1)*(factormonthly[factor])
    
    # Is volatility persistent?
    voldf = factormonthly[['date', 'rolling_var_'+factor, factor]].copy()
    voldf['lag_vol'] = voldf['rolling_var_'+factor].shift(1)
    voldf = voldf.dropna()
    modelvol=sm.OLS(voldf['rolling_var_'+factor], sm.add_constant(voldf[['lag_vol']])).fit()
    print(modelvol.summary())
    
    # Does Vol predict factor returns?
    modelvol=sm.OLS(voldf[factor],sm.add_constant(voldf[['lag_vol']])).fit()
    print(modelvol.summary())
    
    # Asset pricing test of the CAPM
    cols = ['Ret_'+factor, factor]
    returndf = factormonthly[cols].copy().dropna()
    model1=sm.OLS(returndf['Ret_'+factor], sm.add_constant(returndf[[factor]])).fit()
    print(model1.summary())
    
    # AP Test FF3
    cols = ['Ret_'+factor, 'Mkt-RF','SMB', 'HML']
    returndf = factormonthly[cols].copy().dropna()
    model2=sm.OLS(returndf['Ret_'+factor],sm.add_constant(returndf[['Mkt-RF','SMB', 'HML']])).fit()
    print(model2.summary())
    
    # AP Test FF5 + MOM
    cols = ['Ret_'+factor, 'Mkt-RF','SMB', 'HML', 'RMW', 'CMA', 'MOM']
    returndf = factormonthly[cols].copy().dropna()
    model3=sm.OLS(returndf['Ret_'+factor],sm.add_constant(returndf[['Mkt-RF','SMB', 'HML', 'RMW', 'CMA', 'MOM']])).fit()
    print(model3.summary())


                            OLS Regression Results                            
Dep. Variable:     rolling_var_Mkt-RF   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     466.4
Date:                Sun, 01 Oct 2023   Prob (F-statistic):           3.39e-87
Time:                        16:31:02   Log-Likelihood:                -2535.5
No. Observations:                1160   AIC:                             5075.
Df Residuals:                    1158   BIC:                             5085.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5418      0.070      7.790      0.0

                            OLS Regression Results                            
Dep. Variable:        rolling_var_MOM   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.404
Method:                 Least Squares   F-statistic:                     783.2
Date:                Sun, 01 Oct 2023   Prob (F-statistic):          6.06e-132
Time:                        16:31:02   Log-Likelihood:                -1509.2
No. Observations:                1155   AIC:                             3022.
Df Residuals:                    1153   BIC:                             3032.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2097      0.029      7.136      0.0

### Q2 (a) - Aggregate Portfolio

In [53]:
factordaily_res = pd.read_csv('ff6_daily_factors.csv', usecols = [0])

for factor in factorlist:
    factordaily = ffdaily[['date', factor]].copy()
    # Calculate rolling variance
    factordaily['rolling_var_'+factor] = factordaily[factor].rolling(window=22, min_periods=22).var()
    factordaily['inverse_var_'+factor] = 1/ factordaily['rolling_var_'+factor]
    
    # Collapse to Monthly
    factordaily['mdate'] = (factordaily['date']/100).astype(int)
    factordaily = factordaily.drop_duplicates('mdate', keep='last')
    
    factordaily = factordaily[['mdate', 'inverse_var_'+factor, 'rolling_var_'+factor]].copy()
    factordaily = factordaily.rename(columns={'mdate':'date'})
    factordaily_res = pd.merge(factordaily_res, factordaily, on='date', how='right')

factormonthly = pd.merge(ffmonthly, factordaily_res, on='date', how='left')

factormonthly['sum_inv_var'] = factormonthly[['inverse_var_'+ factor for factor in factorlist]].sum(axis=1)

for factor in factorlist:
    factormonthly['weight_'+factor] = factormonthly['inverse_var_'+ factor]/factormonthly['sum_inv_var']
    factormonthly['Ret_'+factor] = factormonthly['weight_'+factor].shift(1)*(factormonthly[factor])

factormonthly['portfolio_return'] = factormonthly[['Ret_'+ factor for factor in factorlist]].sum(axis=1)
factormonthly

,date,Mkt-RF,SMB,HML,RF,MOM,RMW,CMA,inverse_var_Mkt-RF,rolling_var_Mkt-RF,...,Ret_SMB,weight_HML,Ret_HML,weight_RMW,Ret_RMW,weight_CMA,Ret_CMA,weight_MOM,Ret_MOM,portfolio_return
0,192607,2.96,-2.56,-2.43,0.22,NaN,NaN,NaN,4.376654,0.228485,...,NaN,0.323044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,192608,2.64,-1.17,3.82,0.25,NaN,NaN,NaN,2.678081,0.373402,...,-0.520281,0.488781,1.234027,NaN,NaN,NaN,NaN,NaN,NaN,1.326944
2,192609,0.36,-1.40,0.13,0.23,NaN,NaN,NaN,3.925430,0.254749,...,-0.532345,0.458706,0.063541,NaN,NaN,NaN,NaN,NaN,NaN,-0.421653
3,192610,-3.24,-0.09,0.70,0.32,NaN,NaN,NaN,1.445235,0.691929,...,-0.039779,0.368577,0.321094,NaN,NaN,NaN,NaN,NaN,NaN,-0.040440
4,192611,2.53,-0.10,-0.51,0.31,NaN,NaN,NaN,6.535384,0.153013,...,-0.051449,0.267933,-0.187974,NaN,NaN,NaN,NaN,0.114200,NaN,0.056407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,202211,4.60,-3.40,1.38,0.29,-2.01,6.01,3.11,0.314242,3.182266,...,-1.033056,0.125249,0.227922,0.249365,1.130954,0.172719,0.732165,0.030258,-0.144437,1.077022
1157,202212,-6.41,-0.68,1.32,0.33,4.52,0.09,4.19,0.545313,1.833809,...,-0.254384,0.109277,0.165329,0.240928,0.022443,0.206769,0.723694,0.052359,0.136765,0.484153
1158,202301,6.65,5.02,-4.05,0.35,-15.98,-2.62,-4.53,0.855394,1.169051,...,1.713529,0.162961,-0.442570,0.248558,-0.631232,0.195053,-0.936665,0.049611,-0.836689,-0.805606
1159,202302,-2.58,1.21,-0.78,0.34,0.20,0.90,-1.41,0.895622,1.116542,...,0.339789,0.114553,-0.127110,0.385534,0.223702,0.176478,-0.275024,0.037724,0.009922,0.008739


### Q2 (b) - Alphas using CAPM, FF3 and FF5+MOM models on the Aggregate Portfolio

In [54]:
# Asset pricing test of the CAPM
cols = ['portfolio_return', 'Mkt-RF']
returndf = factormonthly[cols].copy().dropna()
model1=sm.OLS(returndf['portfolio_return'],sm.add_constant(returndf['Mkt-RF'])).fit()
print(model1.summary())

# AP Test FF3
cols = ['portfolio_return', 'Mkt-RF','SMB', 'HML']
returndf = factormonthly[cols].copy().dropna()
model2=sm.OLS(returndf['portfolio_return'],sm.add_constant(returndf[['Mkt-RF','SMB', 'HML']])).fit()
print(model2.summary())

# AP Test FF5 + MOM
cols = ['portfolio_return', 'Mkt-RF','SMB', 'HML', 'RMW', 'CMA', 'MOM']
returndf = factormonthly[cols].copy().dropna()
model3=sm.OLS(returndf['portfolio_return'], sm.add_constant(returndf[['Mkt-RF','SMB', 'HML', 'RMW', 'CMA', 'MOM']])).fit()
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:       portfolio_return   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     105.9
Date:                Sun, 01 Oct 2023   Prob (F-statistic):           7.88e-24
Time:                        16:31:16   Log-Likelihood:                -2067.0
No. Observations:                1161   AIC:                             4138.
Df Residuals:                    1159   BIC:                             4148.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3049      0.042      7.175      0.0